In [14]:
import tabula
import pandas as pd
import numpy as np
import pikepdf
from PyPDF2 import PdfFileReader


In [15]:
#count pages
file = "lista-funduszy-ETF.pdf"
pdf = PdfFileReader(open(file,'rb'))
pages = pdf.getNumPages()

In [16]:
# extract urls from pdf
pdf_file = pikepdf.Pdf.open(file)
urls = []
# iterate over PDF pages
for page in pdf_file.pages:
    for annots in page.get("/Annots"):
        uri = annots.get("/A").get("/URI")
        if uri is not None:
            urls.append(str(uri))
url = pd.DataFrame(urls,columns=['url'])
url['Kod ISIN'] = list(map(lambda x: x.split('/')[-1].split('.')[0],url['url']))
url = url.drop_duplicates(subset='Kod ISIN').reset_index(drop=True)
url

,url,Kod ISIN
0,https://www.mdm.pl/ui-pub/site/oferta_indywidu...,etf_dla_zaawansowanych
1,https://www.mdm.pl/addstatic/kid/DE000A0KRJX4.pdf,DE000A0KRJX4
2,https://www.mdm.pl/addstatic/kid/DE000A1N49P6.pdf,DE000A1N49P6
3,https://www.mdm.pl/addstatic/kid/DE000A2BDEB6.pdf,DE000A2BDEB6
4,https://www.mdm.pl/addstatic/kid/GB00B15KXV33.pdf,GB00B15KXV33
...,...,...
279,https://www.mdm.pl/addstatic/kid/IE00BZ048462.pdf,IE00BZ048462
280,https://www.mdm.pl/addstatic/kid/IE00B44CGS96.pdf,IE00B44CGS96
281,https://www.mdm.pl/addstatic/kid/IE00BZ6V7883.pdf,IE00BZ6V7883
282,https://www.mdm.pl/addstatic/kid/IE00BNH72088.pdf,IE00BNH72088


In [17]:
x = []
for i in range(2,pages):
    df = tabula.read_pdf(file,pages=i+1)
    df = pd.DataFrame(df[0])
    #df.columns = df.iloc[0]
    #df = df.rename(columns={np.nan:'Ticker (KID/KIID - Kliknij)'})
    #df = df[df['Nazwa instrumentu (Nazwa w serwisie)']!='Nazwa instrumentu (Nazwa w serwisie)']
    #df = df.dropna()
    x.append(df)
    print('Next page {}'.format(i))
    

Next page 2
Next page 3
Next page 4
Next page 5
Next page 6
Next page 7
Next page 8
Next page 9
Next page 10
Next page 11
Next page 12
Next page 13


In [18]:
# data frame preparation
df = pd.concat(x)
df.columns = ['Nazwa instrumentu', 'Rynek', 'Waluta', 'Kod ISIN',
       'Ticker (KID/KIID)', 'AuM (MLN)', 'Rep.', 'Opl.Zarz.',
       'Dywidenda']
df = df[(df['Nazwa instrumentu']!='Nazwa instrumentu (Nazwa w serwisie)') & (~df['Nazwa instrumentu'].isnull())]


In [21]:
#https://stackoverflow.com/questions/36007849/how-to-split-rows-in-dataframe-based-on-specific-names-in-python
df1 = df.reset_index(drop=True)
df1['Sector'] = df1.iloc[:,0]
sektor = df1['Nazwa instrumentu'][df1['Rynek'].isnull()].tolist()
mask = df1.Sector.isin(sektor)
mask
df1['Sector'] = df1.Sector[mask]
df1['Sector'] = df1.Sector.fillna(method='ffill')
df1= df1[~df1['Rynek'].isnull()]
df1['Dywidenda'] = df1['Dywidenda'].fillna(df1['Opl.Zarz.'])
df1['Opl.Zarz.'] = df1['Opl.Zarz.'].fillna(df1['Dywidenda'])
df1['Dywidenda'] = list(map(lambda x: ' '.join(x.split()[1:]),df1['Dywidenda']))
df1['Opl.Zarz.'] = list(map(lambda x: x.split()[0],df1['Opl.Zarz.']))
df1

,Nazwa instrumentu,Rynek,Waluta,Kod ISIN,Ticker (KID/KIID),AuM (MLN),Rep.,Opl.Zarz.,Dywidenda,Sector
1,ISHARES DIVERSIFIED COMMODITY SWAP UCITS (SXRS...,DEU-XETRA,EUR,IE00BDFL4P12,SXRS:GR,1531,Synt.,"0,19",b. d.,Pozostałe
2,COMSTAGE COMMERZBANK COMMODITY EX-AGRICULTURE ...,DEU-XETRA,EUR,LU0419741177,CBCOMM:GR,318,Synt.,"0,3",Niereg.,Pozostałe
3,ISHARES DIV COMD UCITS (DJCOMEX GR E),DEU-XETRA,EUR,DE000A0H0728,DJCOMEX:GR,280,Synt.,"0,46",Niereg.,Pozostałe
4,DB X-TRACKERS DBLCI COMMODITY OPTIMUM YIELD SW...,DEU-XETRA,EUR,LU0292106167,XDBC:GR,213,Synt.,"0,55",Nie wypłaca,Pozostałe
7,ISHARES STOXX EUROPE 600 AUTOMOBILES & PARTS U...,DEU-XETRA,EUR,DE000A0Q4R28,SXAPEX:GR,321,Fizyczna,"0,46",Kwartalnie,Dóbr wyższego rzędu
...,...,...,...,...,...,...,...,...,...,...
509,HSBC MSCI INDONESIA UCITS (HIDD LN ETF),GBR-LSE,USD,IE00B46G8275,HIDD:LN,47,Fizyczna,"0,6",Niereg.,ETF notowane w USD
510,INVESCO ELWOOD GLOBAL BLOCKCHAIN UCITS (BCHN L...,GBR-LSE,USD,IE00BGBN6P67,BCHN:LN,177,Fizyczna,"0,65",Nie wypłaca,ETF notowane w USD
511,WISDOMTREE SUGAR (SUGA LN ETF),GBR-LSE,USD,GB00B15KY658,SUGA:LN,28,Synt.,"0,49",Nie wypłaca,ETF notowane w USD
512,WISDOMTREE GOLD 3X DAILY LEVERAGED (3GOL LN ETF),GBR-LSE,USD,IE00B8HGT870,3GOL:LN,56,Synt.,"0,99",b. d.,ETF notowane w USD


In [22]:
result = pd.merge(df1,url,on='Kod ISIN')
result.to_csv('lista-funduszy-ETF.csv')